# Toronto Clustering by Enlin Ngo

## Data preparation
### Read wikipage 

In [130]:
import pandas as pd
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(URL) #read wiki page into list using read_html
len(df) #check length
df = df[0] #take first index
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#Preprocessing
Remove row with "not assigned" in 'Borough', replace "not assigned" in 'Neighborhood' with Borough value

In [131]:
#preprocessing
df.rename(columns = {'Postcode':'Postalcode'}, inplace = True)
df = df[df.Borough != 'Not assigned'] #remove borough = not assigned
#replace "not assigned" in neighborhood with borough value
mask = df.Neighborhood == 'Not assigned'
df.loc[mask, 'Neighborhood'] = df.loc[mask, 'Borough']
#double check # df[df.Neighborhood == 'Not assigned']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Group same Postal code neighborhood, add ','

In [132]:
#Grouping same neighboorhood
df_group =df.groupby(['Postalcode','Borough'])['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index()
df_group.shape

(103, 3)

### Look up geo data

In [114]:
#import geo data from csv
geo_dat = pd.read_csv('http://cocl.us/Geospatial_data')
geo_dat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [135]:
#look up value of longtitude & latitude for each postal code
df3 = pd.merge(df_group,geo_dat, left_on ='Postalcode',right_on='Postal Code',how='left')
df3 = df3.drop('Postal Code',axis = 1)
df3.head()


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
